In [ ]:
import sys
sys.path.append('../python_packages_static')
import geopandas as gp
import flopy as fp
import mfsetup
import os
import numpy as np
import matplotlib.pyplot as plt
import yaml
import pyproj



## Using [modflow-setup](https://github.com/aleaf/modflow-setup/tree/develop/mfsetup) it is possible to build only the discretization package to evaluate grid options.

#### get base dir - need this because MODFLOW setup leaves us in the model directory

In [ ]:
basedir = os.getcwd()
basedir

## first get bounding box for the model domain

In [ ]:
domain = gp.read_file('../source_data/shapefiles/Extents/Model_Extent_HUC12.shp')

### get the CRS - we'll need this lateron

In [ ]:
epsg = pyproj.CRS(domain.crs).to_epsg()
epsg

In [ ]:
domain.geometry[0]

In [ ]:
bounds = domain.geometry[0].bounds

In [ ]:
tot_xdist, tot_ydist = bounds[2]+1000-bounds[0], bounds[3]+1000-bounds[1]
print (tot_xdist,tot_ydist)

### set grid spacing and set up grid

In [ ]:
dx = 50
dy = dx

In [ ]:
xcells = int(np.ceil(tot_xdist/dx))
ycells = int(np.ceil(tot_ydist/dy))

In [ ]:
xcells,ycells, xcells*ycells

## Read in the file `example.yml`

In [ ]:
with open('example.yml', 'r') as ifp:
    inyml = yaml.load(ifp, Loader=yaml.FullLoader)

In [ ]:
inyml['setup_grid']['xoff'] = bounds[0]
inyml['setup_grid']['yoff'] = bounds[1]


In [ ]:
inyml['dis']['griddata']['delr'] = dy
inyml['dis']['griddata']['delc'] = dx

In [ ]:
inyml['dis']['dimensions']['nlay'] = 1
inyml['dis']['dimensions']['nrow'] = ycells
inyml['dis']['dimensions']['ncol'] = xcells


In [ ]:
with open('neversink.yml', 'w') as ofp:
    yaml.dump(inyml, ofp)

### now try to make just the DIS package

In [ ]:
m = mfsetup.MF6model.setup_from_yaml('neversink.yml')

In [ ]:
m.dis.idomain.array.sum() * 5

### We can export the grid information as a shapefile to evaluate
NOTE: Errors indicating "No internet connection or epsg code ..." can be safely disregarded as warnings. They are due to restrictive network security preventing access to spatialreference.org but do not impact behavior of these notebooks.

In [ ]:
m.dis.export('neverskink{}m.dis.shp'.format(dx))

In [ ]:
os.chdir(basedir)
